In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_df = pd.read_csv("C:/Users/piyus/Data_Science/Lecture_30/train.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/piyus/Data_Science/Lecture_30/train.csv'

In [ ]:
train_df

# Analysing Missing Data

In [ ]:
train_df.isnull().sum()
#used to check inconsistency in data
#here we can see 177 values from age is missing

In [ ]:
ax = train_df["Age"].hist(bins = 50,color = 'red' , alpha = 0.5)
#alpha means translucency

In [ ]:
ax = train_df["Age"].hist(bins = 50,density = True ,color = 'yellow')
train_df["Age"].plot( kind = 'density' , color = 'blue')

In [ ]:
print(train_df['Embarked'].value_counts())
sns.countplot(x = 'Embarked' , data = train_df)

In [ ]:
train_data = train_df.copy()

# Handling Missing Data

In [ ]:
train_data['Age'].fillna(train_df['Age'].median(),inplace = True)
#inplace is used to do changes on the same address , if not given true then it creates a copy nd then do the changes

In [ ]:
train_df.Embarked.value_counts().idxmax()

In [ ]:
train_data['Embarked'].fillna(train_df.Embarked.value_counts().idxmax() ,inplace = True)
train_data.drop('Cabin' ,axis = 1, inplace = True)
#we dropped Cabin coz only about 20 percent of data was available and we cant recreate the remaining data from this given data

In [ ]:
train_data.isnull().sum()

# Data Analysis and Visualization

In [ ]:
plt.figure(figsize = (15,8)) #used to define the size of graph
ax = train_df["Age"].hist(bins = 20,density = True ,color = 'red' , alpha = 0.5)
train_df["Age"].plot( kind = 'density' , color = 'blue')
ax = train_data["Age"].hist(bins = 20,density = True ,color = 'yellow' , alpha = 0.5)
train_data["Age"].plot( kind = 'density' , color = 'green')
plt.xlim(-20,90)
ax.legend(['Age before' , 'Age after'])

In [ ]:
train_data

In [ ]:
train_data = pd.get_dummies(train_data , columns = ['Sex' , 'Embarked'])
#get dummies jo bhi columns do unko unique values pr tod deta h

In [ ]:
train_data.drop('PassengerId' , axis = 1,inplace = True)
train_data.drop('Name' , axis = 1,inplace = True)
train_data.drop('Ticket' , axis = 1,inplace = True)

In [ ]:
train_data

In [ ]:
train_data['Sex_female'] = np.where(train_data['Sex_female'] == True , 1,0)
train_data['Sex_male'] = np.where(train_data['Sex_male'] == True , 1,0)
train_data['Embarked_C'] = np.where(train_data['Embarked_C'] == True , 1,0)
train_data['Embarked_Q'] = np.where(train_data['Embarked_Q'] == True , 1,0)
train_data['Embarked_S'] = np.where(train_data['Embarked_S'] == True , 1,0)

In [ ]:
train_data

In [ ]:
ax = sns.kdeplot(train_data['Age'][train_data.Survived == 1] , color = 'green' , shade = True)  #kdeplot is used for continous plot
ax = sns.kdeplot(train_data['Age'][train_data.Survived == 0] , color = 'red' , shade = True)
plt.legend(['Survived' , 'Dead'])
plt.xlim(-10,90)

In [ ]:
ax = sns.kdeplot(train_data['Fare'][train_data.Survived == 1] , color = 'green' , shade = True)  #kdeplot is used for continous plot
ax = sns.kdeplot(train_data['Fare'][train_data.Survived == 0] , color = 'red' , shade = True)
plt.legend(['Survived' , 'Dead'])
plt.xlim(-100,200)

In [ ]:
sns.barplot(x = 'Pclass' , y = 'Survived' , data= train_data)

In [ ]:
sns.barplot(x = 'Embarked' , y = 'Survived' , data= train_df)

In [ ]:
sns.barplot(x = 'SibSp' , y = 'Survived' , data= train_df)

In [ ]:
sns.barplot(x = 'Parch' , y = 'Survived' , data= train_df)

In [ ]:
sns.barplot(x = 'Sex' , y = 'Survived' , data= train_df)

In [ ]:
train_data['Travel_Alone'] = np.where((train_data['SibSp'] + train_data['Parch']) >0 ,0 , 1)

In [ ]:
train_data.drop('SibSp' , axis = 1, inplace = True)
train_data.drop('Parch' , axis = 1, inplace = True)

In [ ]:
sns.barplot(x = 'Travel_Alone' , y = 'Survived' , data= train_data)

In [ ]:
plt.figure(figsize = (12,12))
sns.heatmap(train_data.corr() ,annot=True)

In [ ]:
train_data

# Model Training

In [ ]:
cols_model = [ 'Pclass', 'Age' ,'Fare' , 'Sex_female' , 'Sex_male' , 'Embarked_C' , 'Embarked_Q' , 'Embarked_S' , 'Travel_Alone']
train_X = train_data[cols_model]
train_Y = train_data["Survived"]

In [ ]:
print(type(train_X) , type(train_Y))
train_X = train_X.values
train_Y = train_Y.values
print(type(train_X) , type(train_Y))

In [ ]:
split = 0.9
X_train = train_X[:int(split*train_X.shape[0]),:]
Y_train = train_Y[:int(split*train_Y.shape[0])]
X_test = train_X[int(split*train_X.shape[0]):,:]
Y_test = train_Y[int(split*train_Y.shape[0]):]

In [ ]:
print(X_train.shape, Y_train.shape,X_test.shape,Y_test.shape)

In [ ]:
ones = np.ones((X_train.shape[0],1))
X_train = np.hstack((ones,X_train))
ones = np.ones((X_test.shape[0],1))
X_test = np.hstack((ones,X_test))

In [ ]:
print(X_train.shape, Y_train.shape,X_test.shape,Y_test.shape)

In [ ]:
def sigmoid(X):
    return 1/(1+np.exp(-X))

In [ ]:
def hypothesis(X,weights):
    return sigmoid(np.dot(X,weights))

In [ ]:
def loss(X,Y,weights):
    pred = hypothesis(X,weights)
    loss = np.mean(Y* np.log(pred) + (1-Y)*np.log(1-pred))
    return -1*loss

In [ ]:
def gradient(X,Y,weights):
    pred = hypothesis(X,weights)
    grad = np.dot(X.T,(Y-pred))
    return grad/X.shape[0]

In [ ]:
def gradient_descent(X,Y):
    weights = np.zeros((X.shape[1],))
    epochs = 100000
    lr_rate = 0.003
    list_loss = []
    
    for i in range(epochs):
        grad = gradient(X,Y,weights)
        list_loss.append(loss(X,Y,weights))
        weights = weights + (lr_rate*grad)
    return weights,list_loss

In [ ]:
weights , list_loss = gradient_descent(X_train,Y_train)

# Loss Visualization

In [ ]:
list_loss[-1]

In [ ]:
plt.plot(list_loss)

In [ ]:
weights

# Model Testing

In [ ]:
pred = []

for i in range(X_test.shape[0]):
    y_hat = hypothesis(X_test[i],weights)
    print(y_hat , Y_test[i])
    if (y_hat > 0.5):
        pred.append(1)
    else:
        pred.append(0)

In [ ]:
for i in range(Y_test.shape[0]):
    print(pred[i] , '  ' , Y_test[i])

# Calculating Accuracy

In [ ]:
correct = 0
for i in range(Y_test.shape[0]):
    if(pred[i] == Y_test[i]):
        correct += 1
print(correct/Y_test.shape[0])